### This is the same code as what we have been using so far. import mlflow, load iris dataset, split test anbd train data. Then load up different values of the max_iter parameter of the classifier and then under this experiment, run different trainings based on different instances of the classifier. In mlflow UI, you should be able to see these different runs.

In [2]:
import mlflow

mlflow.set_experiment("MLFlow: Search models with code")

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Load the Iris dataset
# X, y = datasets.load_iris(return_X_y=True)
from sklearn.datasets import fetch_openml

wine = fetch_openml("wine-quality-red", version=1, as_frame=True)
X = wine.data
y = (wine.target.astype(int) >= 6).astype(int)
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
max_iterations = [90, 100, 110, 150, 200, 250, 300, 350, 400, 450, 500]

# Enable autologging for scikit-learn
mlflow.sklearn.autolog()

for max_iter in max_iterations:
    params = {
        "solver": "lbfgs",
        "max_iter": max_iter,
        "random_state": 1234,
    }
    lr = LogisticRegression(**params)
    lr.fit(X_train, y_train)

/Users/kowshid/Documents/Repos/grad/ai-545-machine-learning-operations/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2026/02/09 15:49:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/kowshid/Documents/Repos/grad/ai-545-machine-learning-operations/.venv/lib/python3.11/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution becau

### use the search_logged_models that takes as many experiment ids as you want it to have. Get the experiment ID if you click on any run under this experiment on mlflow UI. Here, we are filtering based on metrics_training_f1_score. you can do AND conditions here too if you want. Then we are sorting these models in descending order. and we want the top 5 models. Then select the 0th model as the top model. Extract the model id of that model. BTW, the search returns a pandas dataframe.

In [5]:
top_models = mlflow.search_logged_models(
    experiment_ids=["1"],
    # filter_string="metrics.training_f1_score> 0.97",
    order_by=[{"field_name": "metrics.training_f1_score", "ascending": False}],
    max_results=5,
)
best_model_id = top_models.iloc[0].model_id
loaded_model = mlflow.sklearn.load_model(f"models:/{best_model_id}")
print(f"Best model ID: {best_model_id}")

### Now you got the model id of the best performing model! Just load up the model and use it. No more clicking through the mlflow UI etc! You have automated the model selection now.

In [6]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

X, y = datasets.load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
predictions = loaded_model.predict(X_test)
iris_feature_names = datasets.load_iris().feature_names
result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions
result[:10]